In [1]:
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'

import torch
import torchvision.transforms as T

import lightning.pytorch as pl
import os

In [2]:
# sets seeds for numpy, torch and python.random.
pl.seed_everything(42, workers=True)

Global seed set to 42


42

In [3]:
torch.set_float32_matmul_precision('medium')

# Transforms

In [4]:
train_transform = T.Compose([
    T.RandomCrop((128, 128)),
    T.RandomAdjustSharpness(sharpness_factor=4, p=0.7),
    T.RandomAutocontrast(p=0.3),
    T.ColorJitter(brightness=0.3, hue=0.1, contrast=0.2, saturation=0.4),
    # T.ToTensor(),
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Data Module

In [5]:
from data_module import UdacitySimDataModule

NUM_WORKERS = 1 # os.cpu_count() # <- use all available CPU cores
BATCH_SIZE = 64

udacity_sim_dm = UdacitySimDataModule(
    csv_path_train=".\\data\\driving_train_log.csv",
    csv_path_test=".\\data\\driving_test_log.csv",
    root_dir=".\\data\\",
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transform=train_transform
)

In [6]:
udacity_sim_dm.setup(stage='fit')

In [7]:
train_dataloader = udacity_sim_dm.train_dataloader()
train_features, train_labels = next(iter(train_dataloader))

# for X, y in udacity_sim_dm.train_dataloader():
#     print(X.size())
#     print(y.size())
#     break

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\prana\miniconda3\envs\baby_hamilton\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\prana\miniconda3\envs\baby_hamilton\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\prana\miniconda3\envs\baby_hamilton\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\car practical\baby_hamilton\data_module.py", line 55, in __getitem__
    image = torchvision.io.read_image(self.root_dir + folder + '\\IMG\\' + image)
  File "C:\Users\prana\miniconda3\envs\baby_hamilton\lib\site-packages\torchvision\io\image.py", line 258, in read_image
    data = read_file(path)
  File "C:\Users\prana\miniconda

In [28]:
train_features.size(), train_labels.size()

(torch.Size([32, 3, 128, 128]), torch.Size([32]))

# PyTorch Model

In [12]:
from model import BabyHamiltonModel
model = BabyHamiltonModel()

In [13]:
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_trainable_params

61241

# Lightning Model

In [14]:
from lightning_model import LightningBabyHamiltonModel
lightning_model = LightningBabyHamiltonModel(model)

# Logger

In [15]:
from lightning_model import get_logger
logger = get_logger()

# from lightning.pytorch.loggers import TensorBoardLogger
# logger = TensorBoardLogger(save_dir='logs/', version=1, name="lightning_logs")

# Trainer

In [17]:
trainer = pl.Trainer(
    # Permanent
    default_root_dir="lightning/",
    max_epochs=10,
    devices='auto', # 1, 2, ...
    accelerator="auto", # gpu, cpu, ...
    logger=logger,
    check_val_every_n_epoch=1,
    
    # Temporary
    # fast_dev_run=True,
    # profiler="simple",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Tuner

In [11]:
from lightning.pytorch.tuner import Tuner
tuner = Tuner(trainer)

## Learning Rate

In [ ]:
lr_finder = tuner.lr_find(lightning_model, datamodule=udacity_sim_dm)

In [ ]:
lr_finder.plot(suggest=True)

In [ ]:
new_lr = lr_finder.suggestion()
lightning_model.learning_rate = new_lr
new_lr

## Batch Size

In [ ]:
bs_finder = tuner.scale_batch_size(
    lightning_model, 
    datamodule=udacity_sim_dm
)

In [ ]:
bs_finder.plot(suggest=True)

In [ ]:
new_bs = bs_finder.suggestion()
udacity_sim_dm.batch_size = new_bs
new_bs

# Training

## Fit

In [ ]:
trainer.fit(
    lightning_model, 
    datamodule=udacity_sim_dm
)

## Plot Metrics

In [ ]:
from utils import plot_csv_logger
plot_csv_logger(csv_path=f'{trainer.logger.log_dir}/metrics.csv')

# Testing

In [ ]:
trainer.test(
    lightning_model, 
    datamodule=udacity_sim_dm, 
    ckpt_path='best'
)

# Production

In [30]:
path = trainer.checkpoint_callback.best_model_path
lightning_model = LightningBabyHamiltonModel.load_from_checkpoint(path)

PermissionError: [Errno 13] Permission denied: 'D:/car practical/baby_hamilton'

In [ ]:
x = torch.randn(3, 132, 32)

with torch.inference_mode():
    x = x.unsqueeze(0)
    y_hat = lightning_model(x)